# Testes de acessibilidade

## Introdução

Este notebook conduz testes automatizados de acessibilidade nos portais mapeados no Censo Querido Diário.

Segundo o último Censo Demográfico, há 46 milhões de pessoas no Brasil com algum tipo de deficiência - o equivalente a quase 24% da população<sup>[1](https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/20551-pessoas-com-deficiencia.html)</sup>. Desde 2000, a promoção de mecanismos para garantir o acesso à informação desse segmento da população é uma obrigação do poder público prevista em lei<sup>[2](http://www.planalto.gov.br/ccivil_03/leis/l10098.htm)</sup>.

O objetivo desta análise é verificar se essa obrigação tem sido cumprida pelos municípios brasileiros, no que se refere ao acesso a informações contidas nos diários oficiais. Para isso, buscamos verificar se os portais que dão acesso aos  estão de acordo com os padrões da web para garantir o acesso por pessoas com deficiência.

---

<sup>1</sup>: *IBGE. Conheça o Brasil - População: Pessoas com Deficiência. S/D. Disponível em: <https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/20551-pessoas-com-deficiencia.html>.*

<sup>2</sup>: *BRASIL. Lei Nº 10.098, de 19 de dezembro de 2000. 2000. Disponível em: <http://www.planalto.gov.br/ccivil_03/leis/l10098.htm>.*

## Dependências

Esta análise utiliza um driver de navegador (*geckodrive*) para simular o comportamento de uma pessoa navegando pelos portais dos diários oficiais.

Nas células a seguir, verificamos se o executável desse driver está localizado em algum lugar da raíz do repositório do Censo, no diretório `/notebooks` ou em algum lugar do seu caminho de busca padrão (`$PATH`). Caso não seja encontrado, o programa tentará baixar uma versão do Geckodriver adequada para o seu sistema.

Também são instalados os pacotes [`selenium`](https://pypi.org/project/selenium/) e [`axe-selenium-python`](https://pypi.org/project/axe-selenium-python/) para realizar os testes automatizados. Recomenda-se rodar este notebook em um ambiente virtual para que a instalação desses pacotes não gere conflitos com as dependências instaladas nos sistema.

In [59]:
!pip install -qq requests selenium axe-selenium-python pandas wget

import os
import platform
import shutil
import sys
from pathlib import Path

import wget
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.firefox.options import Options as FirefoxOptions


def download_gecko(version="v0.29.1") -> None:
    # get Gecko executable
    system = platform.system()
    architecture = (int(sys.maxsize > 2**32) + 1) * 32
    gecko_executable_url = (
        f"https://github.com/mozilla/geckodriver/releases/download/{version}/"
        f"geckodriver-{version}-"
    )
    if system == "Windows":
        gecko_executable_url += f"windows{architecture}.zip"
    elif system == "Linux":
        gecko_executable_url += f"linux{architecture}.tar.gz"
    elif system == "Darwin":
        gecko_executable_url += "macos"
        if system.architecture.startswith("arm"):
            gecko_executable_url += "-aarch64"
        gecko_executable_url += ".tar.gz"
    else:
        raise RuntimeError(
            f"No Geckodriver executable available for {system} {architecture}"
        )
    gecko_compressed = wget.download(gecko_executable_url)
    shutil.unpack_archive(gecko_compressed)


# check if geckodriver has been downloaded to the current working directory
driver_options = FirefoxOptions()
driver_options.headless = True
gecko_local_executable = os.path.join(os.getcwd(), "geckodriver")
if Path(gecko_local_executable).is_file():
    executable_path = gecko_local_executable
else:
    executable_path = None

# test creating a new driver; download Gecko if needed
try:
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
except WebDriverException:
    download_gecko()
    executable_path = gecko_local_executable
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
finally:
    driver.close()

## Requisitar dados do Censo Querido Diário

Nesta seção, baixamos os dados mais recentes do mapeamento de portais de diários oficiais do Censo Querido Diário. Em seguida, transformamos as diversas colunas com as URLs das fontes em uma única coluna (formato *longo*).

In [20]:
import pandas as pd

census_data = pd.read_csv("https://censo.ok.org.br/get-data/")
census_data

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_1,fonte_2,fonte_3,fonte_4,is_online,data_inicial,tipo_arquivo,validacao,navegacao,observacoes
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,https://abadiadosdourados.mg.gov.br/novo/index...,None,None,None,1,2013-12-30,PDF texto,True,None,"Desde outubro de 2017, passaram a ser publicad..."
1,Abadia de Goiás (GO),520005,5200050,GO,Região Centro-Oeste,8958,False,None,None,None,None,2,None,None,True,None,Atos publicados de forma avulsa no site da pre...
2,Abaetetuba (PA),150010,1500107,PA,Região Norte,159080,False,http://www.diariomunicipal.com.br/famep/pesqui...,None,None,None,1,2016-06-30,HTML,True,None,NaN
3,Abaré (BA),290020,2900207,BA,Região Nordeste,20347,False,https://sai.io.org.br/ba/abare/site/diariooficial,None,None,None,1,2007-01-09,PDF texto,True,None,None
4,Abaíra (BA),290010,2900108,BA,Região Nordeste,8710,False,http://diariooficial.portalgov.net.br/,None,None,None,1,2017-01-06,PDF imagem,True,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Águas de Lindóia (SP),350050,3500501,SP,Região Sudeste,18808,False,https://imprensaoficialmunicipal.com.br/aguas_...,None,None,None,1,2020-01-31,PDF texto,True,None,NaN
593,Álvares Florence (SP),350120,3501202,SP,Região Sudeste,3647,False,https://imprensaoficialmunicipal.com.br/alvare...,None,None,None,1,2013-03-13,PDF texto,True,None,NaN
594,Álvares Machado (SP),350130,3501301,SP,Região Sudeste,24998,False,http://www.alvaresmachado.sp.gov.br/diariooficial,None,None,None,1,2018-06-04,PDF texto,True,None,NaN
595,Álvaro de Carvalho (SP),350140,3501400,SP,Região Sudeste,5274,False,https://imprensaoficialmunicipal.com.br/alvaro...,None,None,None,1,2018-04-04,PDF texto,True,None,NaN


In [34]:
portals_info = (
    census_data
    .melt(
        id_vars=[
            "municipio",
            "IBGE",
            "IBGE7",
            "UF",
            "regiao",
            "populacao_2020",
            "eh_capital",
        ],
        value_vars=[
            col for col in census_data.columns if col.startswith("fonte_")
        ],
        value_name="fonte",
        var_name="fonte_prioridade",
    )
    .assign(
        fonte_prioridade=lambda _: pd.to_numeric(_.fonte_prioridade.str[-1])
    )
    .replace(to_replace='None', value=pd.NA)
    .dropna(subset=["fonte"])
)

portals_info

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_prioridade,fonte
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...
2,Abaetetuba (PA),150010,1500107,PA,Região Norte,159080,False,1,http://www.diariomunicipal.com.br/famep/pesqui...
3,Abaré (BA),290020,2900207,BA,Região Nordeste,20347,False,1,https://sai.io.org.br/ba/abare/site/diariooficial
4,Abaíra (BA),290010,2900108,BA,Região Nordeste,8710,False,1,http://diariooficial.portalgov.net.br/
5,Abreu e Lima (PE),260005,2600054,PE,Região Nordeste,100346,False,1,http://www.diariomunicipal.com.br/amupe/
...,...,...,...,...,...,...,...,...,...
1364,Cocal do Sul (SC),420425,4204251,SC,Região Sul,16821,False,3,https://www.diariomunicipal.sc.gov.br/site/?r=...
1385,Cuiabá (MT),510340,5103403,MT,Região Centro-Oeste,618124,True,3,https://diariooficial.cuiaba.mt.gov.br/
1440,Guarabira (PB),250630,2506301,PB,Região Nordeste,59115,False,3,https://pmguarabira.wixsite.com/pmguarabira/di...
1611,Pará de Minas (MG),314710,3147105,MG,Região Sudeste,94808,False,3,https://transparencia.parademinas.mg.gov.br/


## Executar a avaliação automatizada dos portais

A seguir, iteramos sobre todos os portais mapeados no Censo Querido Diário, executando os testes automatizados disponibilizados pelo pacote [Axe](https://github.com/dequelabs/axe-core), e registrando as violações aos padrões web de acessibilidade detectadas.

**ATENÇÃO**: Podem ser necessárias algumas horas para realizar a avaliação de todos os portais mapeados no Censo.

In [68]:
import logging

from axe_selenium_python import Axe


logger = logging.getLogger()
logger.setLevel(logging.INFO)


def evaluate_a11y(url: str, driver: webdriver.Firefox) -> pd.DataFrame:
    """Performs an automated acessibility test in the given URL."""
    logger.info(f"Evaluating accessibility in portal <{url}>...")
    driver.get(url)
    axe = Axe(driver)
    # inject axe-core javascript into page.
    axe.inject()
    # run axe accessibility checks.
    results = axe.run()
    # convert into a DataFrame
    df_violations = pd.DataFrame(results["violations"])
    num_violations = len(portal_violations.index)
    num_violations_critical = sum(portal_violations["impact"] == "critical")
    num_violations_serious = sum(portal_violations["impact"] == "serious")
    num_violations_moderate = sum(portal_violations["impact"] == "moderate")
    logger.info(
        f"Found {num_violations} violations "
        f"({num_violations_critical} critical; "
        f"{num_violations_serious} serious; "
        f"{num_violations_moderate} moderate)."
    )
    return df_violations

try:
    # create a new driver
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
    # create a DataFrame to record violations across all portals
    all_violations = pd.DataFrame()
    # evaluate one portal at a time
    num_evaluations = 0
    portal_urls = portals_info["fonte"].unique()
    for url in portal_urls:
        try:
            num_evaluations += 1
            logger.info(
                f"Starting evaluation {num_evaluations} of {len(portal_urls)}."
            )
            portal_violations = evaluate_a11y(url=url, driver=driver)
            portal_violations["fonte"] = url
            all_violations = pd.concat(
                [all_violations, portal_violations],
                ignore_index=True,
            )
        except Exception as err:
            logger.error(f"Error while evaluating <{url}>: {err}")

finally:
    # finish driver
    driver.close()

tarting evaluation 535 of 618.
INFO:root:Evaluating accessibility in portal <https://ioes.dio.es.gov.br/caderno_municipios>...
INFO:root:Found 1 violations (1 critical; 0 serious; 0 moderate).
INFO:root:Starting evaluation 536 of 618.
INFO:root:Evaluating accessibility in portal <https://dom.pmvc.ba.gov.br/>...
INFO:root:Found 8 violations (3 critical; 2 serious; 3 moderate).
INFO:root:Starting evaluation 537 of 618.
INFO:root:Evaluating accessibility in portal <http://www.diariomunicipal.com.br/amupe/pesquisar?busca_avancada[entidadeUsuaria]=46229>...
INFO:root:Found 5 violations (1 critical; 1 serious; 2 moderate).
INFO:root:Starting evaluation 538 of 618.
INFO:root:Evaluating accessibility in portal <https://new.voltaredonda.rj.gov.br/8-interno/658-vr-destaque>...
INFO:root:Found 2 violations (0 critical; 1 serious; 1 moderate).
INFO:root:Starting evaluation 539 of 618.
INFO:root:Evaluating accessibility in portal <https://www.votorantim.sp.gov.br/portal/diario-oficial>...
INFO:root

In [69]:
portals_violations = portals_info.merge(
    all_violations,
    on="fonte",
    how="left",
)

portals_violations

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_prioridade,fonte,description,help,helpUrl,id,impact,nodes,tags
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the contrast between foreground and ba...,Elements must have sufficient color contrast,https://dequeuniversity.com/rules/axe/3.1/colo...,color-contrast,serious,"[{'all': [], 'any': [{'data': {'bgColor': '#ff...","[cat.color, wcag2aa, wcag143]"
1,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures every id attribute value of active ele...,IDs of active elements must be unique,https://dequeuniversity.com/rules/axe/3.1/dupl...,duplicate-id-active,serious,"[{'all': [], 'any': [{'data': 'logo', 'id': 'd...","[cat.parsing, wcag2a, wcag411]"
2,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the order of headings is semantically ...,Heading levels should only increase by one,https://dequeuniversity.com/rules/axe/3.1/head...,heading-order,moderate,"[{'all': [], 'any': [{'data': 3, 'id': 'headin...","[cat.semantics, best-practice]"
3,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures every form element has a label,Form elements must have labels,https://dequeuniversity.com/rules/axe/3.1/labe...,label,critical,"[{'all': [], 'any': [{'data': None, 'id': 'ari...","[cat.forms, wcag2a, wcag332, wcag131, section5..."
4,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the page has only one main landmark an...,Page must have one main landmark,https://dequeuniversity.com/rules/axe/3.1/land...,landmark-one-main,moderate,"[{'all': [{'data': None, 'id': 'page-has-main'...","[cat.semantics, best-practice]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4748,Cocal do Sul (SC),420425,4204251,SC,Região Sul,16821,False,3,https://www.diariomunicipal.sc.gov.br/site/?r=...,Ensures all page content is contained by landm...,All page content must be contained by landmarks,https://dequeuniversity.com/rules/axe/3.1/regi...,region,moderate,"[{'all': [], 'any': [{'data': None, 'id': 'reg...","[cat.keyboard, best-practice]"
4749,Cuiabá (MT),510340,5103403,MT,Região Centro-Oeste,618124,True,3,https://diariooficial.cuiaba.mt.gov.br/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4750,Guarabira (PB),250630,2506301,PB,Região Nordeste,59115,False,3,https://pmguarabira.wixsite.com/pmguarabira/di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4751,Pará de Minas (MG),314710,3147105,MG,Região Sudeste,94808,False,3,https://transparencia.parademinas.mg.gov.br/,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
portals_violations.to_csv("a11y_analysis.csv", index=False)

## Analisar os resultados - Cidades acima de 100 mil habitantes

Nesta seção, analisamos os resultados da avaliação de acessibilidade dos portais **principais** dos municípios com mais de 100 mil habitantes estimados no ano de 2020.

In [ ]:
# TODO